# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import sklearn
import tensorflow as tf
KERAS_BACKEND=tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import gensim
# from keras_self_attention import SeqSelfAttention
# import keras
from keras.utils import np_utils
# from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM
from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.


In [2]:
tf.__version__

'2.3.0'